In [ ]:
# neural network

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Dense
from keras.optimizers import Adam
from keras.initializers import RandomUniform,Constant
from keras import metrics
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import time
from joblib import dump, load

In [3]:
topo_train = pd.read_csv("data/topo_train22.csv")
topo_test_X = pd.read_csv("data/topo_test_X.csv")

topo_train_X = topo_train.iloc[:, :-1]
train_y = topo_train.iloc[:, -1]

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(topo_train_X, train_y, test_size = 0.3, random_state = 0)

In [5]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)

In [6]:
adam = Adam(lr=0.001, beta_1=0.95, beta_2=0.999, epsilon=1e-08)
random_initializer = RandomUniform(minval=-0.005, maxval=0.005)
bias_init = Constant(value=0.001)

In [7]:
# hypermeters objects
batch_size = 10
EPOCHS=30

# define the network
model = Sequential()
model.add(Dense(512, input_dim=x_train.shape[1], use_bias=True, kernel_initializer=random_initializer,
                activation='relu',bias_initializer=bias_init))
model.add(Dropout(0.2))
model.add(Dense(512, use_bias=True, kernel_initializer=random_initializer, activation='relu',
                bias_initializer=bias_init))
model.add(Dropout(0.2))
model.add(Dense(1, use_bias=False, kernel_initializer=random_initializer))
model.add(Dense(1, use_bias=False, kernel_initializer=random_initializer,activation='sigmoid'))


model.compile(optimizer=adam, loss='binary_crossentropy', metrics=["accuracy"])

In [8]:
history = model.fit(x_train,
                    np.array(y_train),
                    shuffle=True,
                    epochs=EPOCHS,
                    batch_size=batch_size,
                    validation_data=(x_valid, y_valid),
                    )

Epoch 1/30
2945/2945 [==============================] - 7s 2ms/step - loss: 0.1040 - accuracy: 0.9587 - val_loss: 0.0630 - val_accuracy: 0.9733
Epoch 2/30
2945/2945 [==============================] - 7s 2ms/step - loss: 0.0300 - accuracy: 0.9902 - val_loss: 0.0181 - val_accuracy: 0.9948
Epoch 3/30
2945/2945 [==============================] - 8s 3ms/step - loss: 0.0076 - accuracy: 0.9979 - val_loss: 0.0021 - val_accuracy: 0.9997
Epoch 4/30
2945/2945 [==============================] - 7s 2ms/step - loss: 0.0038 - accuracy: 0.9991 - val_loss: 0.0134 - val_accuracy: 0.9950
Epoch 5/30
2945/2945 [==============================] - 7s 2ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.0020 - val_accuracy: 0.9992
Epoch 6/30
2945/2945 [==============================] - 6s 2ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0138 - val_accuracy: 0.9976
Epoch 7/30
2945/2945 [==============================] - 7s 2ms/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.0028 - val_accuracy:

In [9]:
x_test = sc.transform(topo_test_X)

predictions = model.predict(x_test)

In [12]:
result = pd.DataFrame({"Id":range(1,2001), "Predicted": predictions.flatten()})
result.to_csv("models/ffnn.csv", index=False)